In [ ]:
!pip install -r ner_example_requirements
!python3 -m spacy download en_core_web_sm

In [ ]:
import os
import pandas as pd
import praw
import spacy
import sqlite3

from hashlib import md5 
from tqdm.notebook import tqdm

# Named Entity Recognition with Spacy

See this [blog post](https://towardsdatascience.com/named-entity-recognition-ner-using-spacy-nlp-part-4-28da2ece57c6) for more details.

In [ ]:
# NER Model
ner_model = spacy.load("en_core_web_sm", 
                       disable=["tok2vec", "tagger", "parser", 
                                "attribute_ruler", "lemmatizer"]
                      )

# Mak example
doc = ner_model("Apple is looking to buy UK start up for $1 billion dollars")
for ent in doc.ents:
    print(ent.text)
    print(ent.label_)
    print(spacy.explain(ent.label_))
    print("-" * 10)

In [ ]:
def get_ner_predictions(text, ner_model, label):
    doc = ner_model(text)
    predictions = [ent.text for ent in doc.ents if ent.label_ == label]
    return predictions

# Extract - Scrape Reddit Data

In order to get your own credentials, please follow the instructions laid out in [this blog post](https://towardsdatascience.com/scraping-reddit-data-1c0af3040768).

In [ ]:
reddit = praw.Reddit(client_id=os.environ["REDDIT_CLIENT_ID"],
                     client_secret=os.environ["REDDIT_CLIENT_SECRET"],
                     user_agent=os.environ["REDDIT_USER_AGENT"],
                     username=os.environ["REDDIT_USERNAME"],
                     password=os.environ["REDDIT_PASSWORD"])

In [ ]:
subreddit = reddit.subreddit("nyc")

top_posts = list(subreddit.top(limit=50))

comments = []
users = []
title = []
submission_id = []
time = []
for submission in tqdm(top_posts):
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if not comment.author:
            continue
        comments.append(comment.body)
        users.append(comment.author.name)
        title.append(submission.title)
        submission_id.append(submission.name)
        time.append(comment.created_utc)

In [ ]:
# Make dataframe and persist data
extracted_reddit_data = pd.DataFrame(
    {'comment': comments,
     'user': users,
     'post': title,
     'time': time}
)
extracted_reddit_data['hash'] = extracted_reddit_data.comment.apply(lambda x: md5(x.encode()).hexdigest())
extracted_reddit_data.to_csv("extracted_reddit_data_nyc_2021_01_17.csv", index=False)

In [ ]:
# Load Cached Data
extracted_reddit_data = pd.read_csv("extracted_reddit_data_nyc_2021_01_17.csv")

# Transform - Get NER Predictions

In [ ]:
predicted_entities, users, hashes = [], [], []

for _, row in tqdm(extracted_reddit_data.iterrows(), 
                   total=len(extracted_reddit_data)):
    text = row['comment']
    user = row['user']
    text_hash = md5(text.encode()).hexdigest()
    text_entity_predictions = get_ner_predictions(text, ner_model, label='GPE')
    if text_entity_predictions:
        predicted_entities += text_entity_predictions
        users += [user] * len(text_entity_predictions)
        hashes += [text_hash] * len(text_entity_predictions)

In [ ]:
# Make dataframe and persist data
transformed_reddit_data = pd.DataFrame({'user': users, 
                                        'predicted_entity': predicted_entities,
                                        'hash': hashes})
transformed_reddit_data.to_csv("transformed_reddit_data_nyc_2021_01_17.csv", index=False)

# Load Data into SQL Database

Browse your a local Sqlite database using [DBrowser](https://sqlitebrowser.org/dl/)

## Extracted Data

In [ ]:
con = sqlite3.connect('reddit.db')
cursor = con.cursor()
print("Successfully Connected to Database")

# Drop table if already exists
cursor.execute("DROP TABLE IF EXISTS comments")
con.commit()

# Create Table
sql_create_table = """CREATE TABLE IF NOT EXISTS comments (
                                    table_id integer PRIMARY KEY,
                                    user text NOT NULL,
                                    comment text NOT NULL,
                                    hash text NOT NULL,
                                    time integer NOT NULL
                                );"""
cursor.execute(sql_create_table)
con.commit()
print("Succesfully create comments table")

# Insert rows from extracted entity dataframe
for i, row in tqdm(extracted_reddit_data.iterrows(), 
                   total=len(extracted_reddit_data)):
    sqlite_insert_query = f"""INSERT INTO comments
                          (table_id, user, comment, hash, time) 
                           VALUES 
                          ('{i}', '{row['user']}', 
                          '{row['comment'].replace("'", "''")}', '{row['hash']}', '{row['time']}')"""

    cursor.execute(sqlite_insert_query)
    con.commit()
print(f"Successfully inserted {len(extracted_reddit_data)} rows into the comments table")
cursor.close()

## Predicted Data

In [ ]:
con = sqlite3.connect('reddit.db')
cursor = con.cursor()
print("Successfully Connected to Database")

# Drop table if already exists
cursor.execute("DROP TABLE IF EXISTS entity_predictions")
con.commit()

# Create Table
sql_create_table = """CREATE TABLE IF NOT EXISTS entity_predictions (
                                    table_id integer PRIMARY KEY,
                                    user text NOT NULL,
                                    predicted_entity text NOT NULL,
                                    hash text NOT NULL
                                );"""
cursor.execute(sql_create_table)
con.commit()
print("Succesfully create entity_predictions table")

# Insert rows from extracted entity dataframe
for i, row in tqdm(transformed_reddit_data.iterrows(), 
                   total=len(transformed_reddit_data)):
    sqlite_insert_query = f"""INSERT INTO entity_predictions
                          (table_id, user, predicted_entity, hash) 
                           VALUES 
                          ('{i}', '{row['user']}', '{row['predicted_entity'].replace("'", "''")}', '{row['hash']}')"""

    cursor.execute(sqlite_insert_query)
    con.commit()
print(f"Successfully inserted {len(transformed_reddit_data)} rows into the entity_predictions table")
cursor.close()

# Connected Components Example

In [ ]:
con = sqlite3.connect('reddit.db')
cursor = con.cursor()

# Drop table if already exists
bushwick_comments_query = """
SELECT
  c.user,
  c.comment
FROM entity_predictions ep
JOIN comments c
  ON ep.hash = c.hash
WHERE ep.predicted_entity = 'Bushwick'
"""
bushwick_mentions = pd.read_sql(bushwick_comments_query, con=con)
cursor.close()
for _, row in bushwick_mentions.iterrows():
    print(f"User: {row['user']}")
    print(f"Comment: {row['comment']}")
    print("-" * 10)